## In this notebook we will explore what data is offered by Copernicus Satellites Sentinel-3 OLCI

More detailed information can be found here: https://scihub.copernicus.eu/userguide/

The user guide can be found here: https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-3-altimetry

And the technical guide here: https://sentinel.esa.int/web/sentinel/technical-guides/sentinel-3-olci

## Sentinel 3 classification


Marine data is often not available from copernicus open access hub directly, but can be found here: https://marine.copernicus.eu/


The Spacecraft carries four main instruments:
- OLCI: Ocean and Land Colour Instrument
- SLSTR: Sea and Land Surface Temperature Instrument
- SRAL: SAR Radar Altimeter
- MWR: Microwave Radiometer.


Sentinel-3's Data Product is more Complex than the others, so user products are distributed in Product Dissemination Units (PDU), there are three defined types: 
- frame: Identified by a fixed coordinate system based in along-track coordinates and along-orbit cycle coordinates. Each frame is stepped by a constant time interval along this orbit.
- stripe: A stripe either coincides with an acquisition dump or a defined time segment (one orbit, half an orbit,..). A special case is the Vegetation like product (VGT-P) which is a stripe mapped on a Plate Carrée geographical projection.
- tile: Tiles are only defined for VGT 1 and 10 day synthesis product's, VGT-S1 (SY_2_VG1) and VGT-S10 (SY_2_V10)

The general product structure is:

[![Alt text](./images/S3_Product_Structure.jpg)](https://sentinels.copernicus.eu/web/sentinel/missions/sentinel-3/data-products)



## Accessible data from Sentinel-3 OLCI:

- Level-1B: provides radiances for each pixel in the instrument grid, each view and each OLCI channel, plus annotation data associated to OLCI pixels.
    - OL_1_EFR: output during Earth Observation processing mode for Full Resolution at 300m spatial resolution
    - OL_1_ERR: output during Earth Observation processing mode for Reduced Resolution at 1.2km spatial resolution
- Level-2 Land: provides land and atmospheric geophysical parameters computed for full and Reduced Resolution.
    - OL_2_LFR: Full Resolution at 300m spatial resolution
    - OL_2_LRR: Reduced Resolution at 1.2km spatial resolution
- Level-2 Water: provides water and atmospheric geophysical parameters computed for Full and Reduced Resolution.
    - OL_2_WFR: Full Resolution at 300m spatial resolution
    - OL_2_WRR: Reduced Resolution at 1.2km spatial resolution

These can be withdrawn in two different timelines:
- Near Real Time (NRT): delivery less than 3 hours after data acquisition
- Non-Time Crititcal (NTC): delivery within 1 month after data acquisition

These are respectively identified by the NR and NT suffix in their filename 



## Accessible data from Sentinel-3 SLSTR
- Level-1B: provides radiance and brightness temperatures for each pixel in an image grid for each SLSTR channel and view
- Level-2:
    - WST: L2P sea surface temperature according to GHRSST specifications
    - LST: Land surface temperatures
    - FRP: Provides fire radiative power over land and water
    - AOD: Provides aerosol optical depth over land and water

These can be withdrawn in two different timelines:
- Near Real Time (NRT): delivery less than 3 hours after data acquisition
- Non-Time Crititcal (NTC): delivery within 1 month after data acquisition

## Accessible data from Sentinel-3 Synergy
The Syn products are a product of the OLCI and the SLSTR data

- Level 2:
    - SY_2_SYN__: Surface reflectance and aerosol parameters over land
    - SY_2_VGP__: 1km Vegetation Like Product and Top of Atmosphere refelctance
    - SY_2_VGK__: Surface reflectance over land (input for VG-S product)
    - SY_2_VG1__: 1km Vegetation Like Product 1 day synthesis surface reflectance and NDVI
    - SY_2_V10__: 1km Vegetation Like Product 10 day synthesis surface reflectance and NDVI
    - SY_2_AOD__: Global Aerosol parameter over land and sea on super pixel resolution (4.5km x 4.5km)

## Accessible data from Sentinel-3 Altimetry

- Level 1A: geo-located bursts of echo with callibrations applied
- Level 1B: geo-located and fully calibrated multi-looked High Resolution power echoes
- Level 1B-S: SAR processed and calibrated High resolution complex echoes arranged in stacks after slant range correction and prior to echo multi-look
- Level 2: SRAL/MWR

[![Alt text](./images/s3_SRAL_MWR.jpeg)](https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-3-altimetry/product-types/level-2-sral-mwr)



All identifiers for the querry can be found here https://scihub.copernicus.eu/twiki/do/view/SciHubUserGuide/FullTextSearch?redirectedfrom=SciHubUserGuide.3FullTextSearch

Within it you can also find the naming conventions

## Sentinel-3 OLCI

In [3]:
# Here we will retrieve one example of each offered product level and display its quick look image
from sentinelsat import SentinelAPI
user = ''
password = ''

api = SentinelAPI(user, password, 'https://apihub.copernicus.eu/apihub')

In [5]:
# Level 1B: EFR 

# This querry will look for products from the Sentinel 3A satelite S3A, OL refers to OLCI, 1 refers to the product level, next is the Data Type ID EFR___, and then again year and wildcard to narrow down the search
products = api.query(platformname = 'Sentinel-3',
                     filename='S3A_OL_1_EFR____2021*')

# We will use sort according to size and then download the first dataset that is not in the Long Term Archive (These must be requested 30 min in advance)
try:
    df = api.to_dataframe(products).sort_values(['size'], ascending=[True])
except:
    df = api.to_dataframe(products)
    print('Size not specified')
for i in range(len(df)):
    #Check if resource is online
    if api.is_online('{}'.format(df.index[i])):
        api.download('{}'.format(df.index[i]))
        #And break the i loop
        break
    else: pass

print('The File Name is {}'.format(df.iloc[i]['filename']))

The File Name is S3A_OL_1_EFR____20210922T161044_20210922T161044_20210922T183015_0000_076_311_1440_LN1_O_NR_002.zip


In [4]:
import zipfile

with zipfile.ZipFile('S3A_OL_1_EFR____20210922T161044_20210922T161044_20210922T183015_0000_076_311_1440_LN1_O_NR_002.SEN3'.replace('.SEN3','.zip')) as z:
    z.extractall()

In [5]:
# To open the netCDF files (nc), note that to open a file we have to specify mode 'r'
from netCDF4 import Dataset
rootgrp = Dataset('S3A_OL_1_EFR____20210922T161044_20210922T161044_20210922T183015_0000_076_311_1440_LN1_O_NR_002.SEN3/Oa01_radiance.nc', "r")


In [6]:
rootgrp

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    absolute_orbit_number: 29159
    ac_subsampling_factor: 64
    al_subsampling_factor: 1
    comment:  
    contact: eosupport@copernicus.esa.int
    creation_time: 2021-09-22T18:30:15Z
    history:   2021-09-22T18:30:15Z: PUGCoreProcessor /data/ipf-s3/workdir154/635310732/JobOrder.635310732.xml
    institution: LN1
    netCDF_version: 4.2 of Mar 13 2018 10:14:33 $
    product_name: S3A_OL_1_EFR____20210922T161044_20210922T161044_20210922T183015_0000_076_311_1440_LN1_O_NR_002.SEN3
    references: S3IPF PDS 004.1 - i2r4 - Product Data Format Specification - OLCI Level 1, S3IPF PDS 002 - i1r7 - Product Data Format Specification - Product Structures, S3IPF DPM 002 - i2r5 - Detailed Processing Model - OLCI Level 1
    resolution: [ 270 294 ]
    source: IPF-OL-1-EO 06.11
    start_time: 2021-09-22T16:10:43.858174Z
    stop_time: 2021-09-22T16:10:43.946173Z
    title: OLCI Level 1b Product, Radiance Oa0

In [8]:
#To extract more information about a variable we simply do:
rootgrp['Oa01_radiance']

<class 'netCDF4._netCDF4.Variable'>
uint16 Oa01_radiance(rows, columns)
    _FillValue: 65535
    add_offset: 0.0
    ancillary_variables: Oa01_radiance_err
    coordinates: time_stamp altitude latitude longitude
    long_name: TOA radiance for OLCI acquisition band Oa01
    scale_factor: 0.01394646
    standard_name: toa_upwelling_spectral_radiance
    units: mW.m-2.sr-1.nm-1
    valid_max: 65534
    valid_min: 0
unlimited dimensions: 
current shape = (3, 4865)
filling on

In [11]:
#And to get the data
data = rootgrp['Oa01_radiance'][:]
type(data)

numpy.ma.core.MaskedArray

In [22]:
# Level 2: LFR 

# This querry will look for products from the Sentinel 3A  satelite S3A, from level 2,LFR___ data and then year to reduce the output and a wildcard *
products = api.query(platformname = 'Sentinel-3',
                     filename='S3A_OL_2_LFR____2020*')

# We will use sort according to size and then download the first dataset that is not in the Long Term Archive (These must be requested 30 min in advance)
try:
    df = api.to_dataframe(products).sort_values(['size'], ascending=[True])
except:
    df = api.to_dataframe(products)
    print('Size not specified')
for i in range(len(df)):
    #Check if resource is online
    try:
        if api.is_online('{}'.format(df.index[i])):
            api.download('{}'.format(df.index[i]))
            #And break the i loop
            break
        else: pass
    except Exception as e:
        print(e)

print('The File Name is {}'.format(df.iloc[i]['filename']))

Invalid key (a01c6a4c-3b7f-4805-b1a0-f24067c36756) to access Products


The File Name is S3A_OL_2_LFR____20201219T091955_20201219T092255_20201220T141255_0179_066_207_2340_LN1_O_NT_002.SEN3


In [15]:
import zipfile

with zipfile.ZipFile('S3A_OL_2_LFR____20201219T091955_20201219T092255_20201220T141255_0179_066_207_2340_LN1_O_NT_002.SEN3'.replace('.SEN3','.zip')) as z:
    z.extractall()

To retrieve data I will refer you back to the  S3A_OL_1_EFR exmple (the first OLCI dataset)

## Sentinel-3 SLSTR

To reduce the amount of data in this notebook we will skip the 1B Product and go straight to the level 2 products

- WST: L2P sea surface temperature according to GHRSST specifications (can be retrieved from the Copernicus marine hub)
- LST: Land surface temperatures
- FRP: Provides fire radiative power over land and water
- AOD: Provides aerosol optical depth over land and water



In [13]:
# Level 2: LST

# This querry will look for products from the Sentinel 3A and 3B satelite S3_, from level 2, LST__ data, a year to reduce the output and a wildcard *
products = api.query(platformname = 'Sentinel-3',
                     filename='S3*_SL_2_LST____2021*')

# We will use sort according to size and then download the first dataset that is not in the Long Term Archive (These must be requested 30 min in advance)
try:
    df = api.to_dataframe(products).sort_values(['size'], ascending=[True])
except:
    df = api.to_dataframe(products)
    print('Size not specified')
for i in range(len(df)):
    #Check if resource is online
    if api.is_online('{}'.format(df.index[i])):
        api.download('{}'.format(df.index[i]))
        #And break the i loop
        break
    else: pass

print('The File Name is {}'.format('S3A_SL_2_LST____20210930T054219_20210930T054221_20210930T083707_0001_077_034_0540_LN2_O_NR_004.SAFE')) #df.iloc[i]['filename']

The File Name is S3A_SL_2_LST____20210930T054219_20210930T054221_20210930T083707_0001_077_034_0540_LN2_O_NR_004.SAFE


In [16]:
import zipfile

with zipfile.ZipFile('S3A_SL_2_LST____20210930T054219_20210930T054221_20210930T083707_0001_077_034_0540_LN2_O_NR_004.SEN3'.replace('.SEN3','.zip')) as z:
    z.extractall()

To retrieve data I will refer you back to the  S3A_OL_1_EFR exmple (the first OLCI dataset)

In [18]:
# Level 2: FRP

# This querry will look for products from the Sentinel 3A and 3B satelite S3_, from level 2, FRP__ data, a year to reduce the output and a wildcard *
products = api.query(platformname = 'Sentinel-3',
                     filename='S3A_SL_2_FRP____2021*')

# We will use sort according to size and then download the first dataset that is not in the Long Term Archive (These must be requested 30 min in advance)
try:
    df = api.to_dataframe(products).sort_values(['size'], ascending=[True])
except:
    df = api.to_dataframe(products)
    print('Size not specified')
for i in range(len(df)):
    #Check if resource is online
    if api.is_online('{}'.format(df.index[i])):
        api.download('{}'.format(df.index[i]))
        #And break the i loop
        break
    else: pass

print('The File Name is {}'.format(df.iloc[i]['filename']))


















































































































































































































































































Querying products:   0%|          | 400/151346 [1:38:20<824:43:26, 19.67s/product]






































































































































































































































































































































































































































































































































































































































































The File Name is S3A_SL_2_FRP____20210717T112022_20210717T112114_20210817T223957_0052_074_123_0000_LN2_O_NT_004.SEN3


In [19]:
import zipfile #TODO:

with zipfile.ZipFile('S3A_SL_2_FRP____20210717T112022_20210717T112114_20210817T223957_0052_074_123_0000_LN2_O_NT_004.SEN3'.replace('.SEN3','.zip')) as z:
    z.extractall()

To retrieve data I will refer you back to the  S3A_OL_1_EFR exmple (the first OLCI dataset)

In [22]:
# Level 2: AOP

# This querry will look for products from the Sentinel 3A and 3B satelite S3_, from level 2, FRP__ data, a year to reduce the output and a wildcard *
products = api.query(platformname = 'Sentinel-3',
                     filename='S3*_SL_2_AOP____2021*')

# We will use sort according to size and then download the first dataset that is not in the Long Term Archive (These must be requested 30 min in advance)
try:
    df = api.to_dataframe(products).sort_values(['size'], ascending=[True])
except:
    df = api.to_dataframe(products)
    print('Size not specified')
for i in range(len(df)):
    #Check if resource is online
    if api.is_online('{}'.format(df.index[i])):
        api.download('{}'.format(df.index[i]))
        #And break the i loop
        break
    else: pass

print('The File Name is {}'.format(df.iloc[i]['filename']))

Size not specified


NameError: name 'i' is not defined

In [ ]:
#TODO:

## Sentinel 3 Synergy

- Level 2:
    - SY_2_SYN__: Surface reflectance and aerosol parameters over land
    - SY_2_VGP__: 1km Vegetation Like Product and Top of Atmosphere refelctance
    - SY_2_V10__: 1km Vegetation Like Product 10 day synthesis surface reflectance and NDVI
    - SY_2_AOD__: Global Aerosol parameter over land and sea on super pixel resolution (4.5km x 4.5km)


In [ ]:
# Level 2: SY_2_SYN__

# This querry will look for products from the Sentinel 3A and 3B satelite S3_, from level 2, SYN__ data, a year to reduce the output and a wildcard *
products = api.query(platformname = 'Sentinel-3',
                     filename='S3__SY_2_SYN____2021*')

# We will use sort according to size and then download the first dataset that is not in the Long Term Archive (These must be requested 30 min in advance)
try:
    df = api.to_dataframe(products).sort_values(['size'], ascending=[True])
except:
    df = api.to_dataframe(products).sort_values(['size'], ascending=[True])
    print('Size not specified')
for i in range(len(df)):
    #Check if resource is online
    if api.is_online('{}'.format(df.index[i])):
        api.download('{}'.format(df.index[i]))
        #And break the i loop
        break
    else: pass

print('The File Name is {}'.format(df.iloc[i]['filename']))

In [14]:
#TODO:

In [ ]:
# Level 2: SY_2_VGP__

# This querry will look for products from the Sentinel 3A and 3B satelite S3_, from level 2, VGP__ data, a year to reduce the output and a wildcard *
products = api.query(platformname = 'Sentinel-3',
                     filename='S3__SY_2_VGP____2021*')

# We will use sort according to size and then download the first dataset that is not in the Long Term Archive (These must be requested 30 min in advance)
try:
    df = api.to_dataframe(products).sort_values(['size'], ascending=[True])
except:
    df = api.to_dataframe(products).sort_values(['size'], ascending=[True])
    print('Size not specified')
for i in range(len(df)):
    #Check if resource is online
    if api.is_online('{}'.format(df.index[i])):
        api.download('{}'.format(df.index[i]))
        #And break the i loop
        break
    else: pass

print('The File Name is {}'.format(df.iloc[i]['filename']))

In [ ]:
#TODO:

In [ ]:
# Level 2: SY_2_V10__

# This querry will look for products from the Sentinel 3A and 3B satelite S3_, from level 2, V10__ data, a year to reduce the output and a wildcard *
products = api.query(platformname = 'Sentinel-3',
                     filename='S3__SY_2_V10____2021*')

# We will use sort according to size and then download the first dataset that is not in the Long Term Archive (These must be requested 30 min in advance)
try:
    df = api.to_dataframe(products).sort_values(['size'], ascending=[True])
except:
    df = api.to_dataframe(products).sort_values(['size'], ascending=[True])
    print('Size not specified')
for i in range(len(df)):
    #Check if resource is online
    if api.is_online('{}'.format(df.index[i])):
        api.download('{}'.format(df.index[i]))
        #And break the i loop
        break
    else: pass

print('The File Name is {}'.format(df.iloc[i]['filename']))

In [ ]:
#TODO:

In [ ]:
# Level 2: SY_2_AOD__

# This querry will look for products from the Sentinel 3A and 3B satelite S3_, from level 2, AOD__ data, a year to reduce the output and a wildcard *
products = api.query(platformname = 'Sentinel-3',
                     filename='S3__SY_2_AOD____2021*')

# We will use sort according to size and then download the first dataset that is not in the Long Term Archive (These must be requested 30 min in advance)
try:
    df = api.to_dataframe(products).sort_values(['size'], ascending=[True])
except:
    df = api.to_dataframe(products).sort_values(['size'], ascending=[True])
    print('Size not specified')
for i in range(len(df)):
    #Check if resource is online
    if api.is_online('{}'.format(df.index[i])):
        api.download('{}'.format(df.index[i]))
        #And break the i loop
        break
    else: pass

print('The File Name is {}'.format(df.iloc[i]['filename']))

In [ ]:
#TODO:

## Sentinel-3 Altimetry

- Level 1A: geo-located bursts of echo with callibrations applied
- Level 1B: geo-located and fully calibrated multi-looked High Resolution power echoes
- Level 1B-S: SAR processed and calibrated High resolution complex echoes arranged in stacks after slant range correction and prior to echo multi-look
- Level 2: SRAL/MWR

In [ ]:
# Level 1

# This querry will look for products from the Sentinel 3A and 3B satelite S3_, from level 1, SRA__ for LRM, SAR Ku and SAR C products, a year to reduce the output and a wildcard *,
# here we can also specify which product generating center was used, we will withdraw the marine data with MAR at the correct indexer 
products = api.query(platformname = 'Sentinel-3',
                     filename='S3__SS_1_AOD____2021*_*_*_*_MAR_*')

# We will use sort according to size and then download the first dataset that is not in the Long Term Archive (These must be requested 30 min in advance)
try:
    df = api.to_dataframe(products).sort_values(['size'], ascending=[True])
except:
    df = api.to_dataframe(products).sort_values(['size'], ascending=[True])
    print('Size not specified')
for i in range(len(df)):
    #Check if resource is online
    if api.is_online('{}'.format(df.index[i])):
        api.download('{}'.format(df.index[i]))
        #And break the i loop
        break
    else: pass

print('The File Name is {}'.format(df.iloc[i]['filename']))

In [ ]:
#TODO:

In [ ]:
#Level 2

# This querry will look for products from the Sentinel 3A and 3B satelite S3_, from level 2, LAN__ for SRAL land products, a year to reduce the output and a wildcard *,
products = api.query(platformname = 'Sentinel-3',
                     filename='S3__SS_2_SRA____2021*')

# We will use sort according to size and then download the first dataset that is not in the Long Term Archive (These must be requested 30 min in advance)
try:
    df = api.to_dataframe(products).sort_values(['size'], ascending=[True])
except:
    df = api.to_dataframe(products).sort_values(['size'], ascending=[True])
    print('Size not specified')
for i in range(len(df)):
    #Check if resource is online
    if api.is_online('{}'.format(df.index[i])):
        api.download('{}'.format(df.index[i]))
        #And break the i loop
        break
    else: pass

print('The File Name is {}'.format(df.iloc[i]['filename']))


In [ ]:
#TODO: